<a href="https://colab.research.google.com/github/Sophiajackrich/Stutern-repo/blob/main/SQL_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2 (Week 2)

>**Note**: Late submissions are ***`penalized`***.

## Name: JACKRICH SOPHIA

In [13]:
import pandas as pd
# Sql library
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine

### The Database can be found [here](https://drive.google.com/file/d/1uIPkJWIHyZ8iQD_tQlM_K-ANoT_Bpy8P/view?usp=sharing)

> Run the command below to see all the tables present in the SQLite DB.

```sql
SELECT name FROM sqlite_master;
```

<hr>
<br>

In [14]:
sqlite_con = sqlite3.connect("week_2.db")

## Qs 1: 

a. Given the table, `customers` and `orders` shown below, use a **subquery** to find the **customers** that did ***NOT*** place any orders. Display only the customer name(s).

**`customers`**

![image.png](https://i.postimg.cc/sx2WK6v3/image.png)


**`orders`**

![image.png](https://i.postimg.cc/T2H5fcKW/image.png)

b. Given the `products` table shown below:

[![image.png](https://i.postimg.cc/W18dpLXC/image.png)](https://postimg.cc/DSWv5DVP)

write a query that returns the price of each product at every store. Your result should look like this:

[![image.png](https://i.postimg.cc/s235K4Nd/image.png)](https://postimg.cc/FYnfH3vD)

**Hint:** Use `CASE` statement.

### NOTE: You must use a single query.

In [15]:
pd.read_sql("SELECT * FROM sqlite_master;", sqlite_con)


,type,name,tbl_name,rootpage,sql
0,table,customers,customers,2,CREATE TABLE customers (\n id INTEG...
1,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
2,table,orders,orders,4,CREATE TABLE orders (\n id INTEGER ...
3,table,employee,employee,5,CREATE TABLE employee(\n id INTEGER...
4,table,department,department,6,CREATE TABLE department (\n id INTE...
5,table,students,students,7,CREATE TABLE students(\n matric_no ...
6,index,sqlite_autoindex_students_1,students,8,None
7,table,products,products,9,"CREATE TABLE products(\n id INT,\n ..."


1a. Given the table, `customers` and `orders` shown below, use a **subquery** to find the **customers** that did ***NOT*** place any orders. Display only the customer name(s).

In [16]:
pd.read_sql("SELECT * FROM customers;", sqlite_con)


,id,name
0,1,Mike
1,2,Afam
2,3,Deborah
3,4,Yusuf


In [17]:
pd.read_sql("SELECT * FROM orders;", sqlite_con)

,id,order_id
0,1,5.0
1,2,NaN
2,3,NaN
3,4,3.0


In [31]:
# 1 a. Given the table, 
# customers and orders shown below, use a subquery to find the customers that did NOT place any orders. Display only the customer name(s).
pd.read_sql("""WITH t1 AS(SELECT id, order_id
                          FROM orders
                          WHERE order_id IS NULL)
               SELECT c.name
               FROM customers c
               JOIN t1 
               USING(id)""", sqlite_con)


,name
0,Afam
1,Deborah


### 1 b. write a query that returns the price of each product at every store. Your result should look like this:

In [32]:
pd.read_sql("""
            SELECT id, MAX(store_a) AS store_a, MAX(store_b) AS store_b, MAX(store_c) AS store_c
            FROM (SELECT *, Row_number() OVER (PARTITION BY store ORDER BY price) AS str_p,
                    CASE 
                        WHEN store = 'store a' THEN price
                        END AS store_a,
                    CASE 
                        WHEN store = 'store b' THEN price
                        END AS store_b,
                    CASE 
                        WHEN store = 'store c' THEN price
                        END AS store_c
                FROM products) t1
                GROUP BY str_p
               """, sqlite_con)

,id,store_a,store_b,store_c
0,0,10,11,12
1,1,33,35,34


In [20]:
# OR 1b.
pd.read_sql("""SELECT id,
                    CASE 
                        WHEN store = 'store a' THEN price
                        END AS store_a,
                    CASE 
                        WHEN store = 'store b' THEN price
                        END AS store_b,
                    CASE 
                        WHEN store = 'store c' THEN price
                        END AS store_c
             FROM products
             """, sqlite_con)

,id,store_a,store_b,store_c
0,0,10.0,NaN,NaN
1,0,NaN,NaN,12.0
2,1,NaN,35.0,NaN
3,1,33.0,NaN,NaN
4,1,NaN,NaN,34.0
5,0,NaN,11.0,NaN


## Qs 2: 

Given the tables `employee` and `department` shown below:

![image.png](https://i.postimg.cc/bw0J574p/image.png)

a. Write a query that generates the table shown below:

![image](https://i.postimg.cc/hjc3rdTK/image.pnghttps://i.postimg.cc/hjc3rdTK/image.png)

b. Write a query that determines the **highest earner** in **each department**.

c. Write a query that shows all the employees in each department **except** the **lowest earner**. i.e generate the table below

![image.png](https://i.postimg.cc/MHcmw7Q0/image.png)

In [33]:
# 2a: Write a query that generates the table shown below:
pd.read_sql("""SELECT name, salary, dept_name
               FROM(SELECT *,Row_number() OVER (PARTITION BY dept_name) t1
                      FROM employee e
                      JOIN department d
                      ON e.dept_id = d.id) t1
               ;""", sqlite_con)

,name,salary,dept_name
0,Felix,33000,IT
1,Okafor,25000,IT
2,Gafar,41000,IT
3,Amanda,53000,IT
4,Steve,32000,Sales
5,Bola,31000,Sales
6,Babalola,30200,Sales
7,Adigun,30000,Sales


In [34]:
# 2b: Write a query that determines the highest earner in each department.
pd.read_sql("""SELECT name, salary, dept_name, rank
               FROM (SELECT name, dept_name, salary, ROW_NUMBER() OVER (PARTITION BY dept_name ORDER BY salary DESC) rank
                       FROM employee e
                       JOIN department d
                       ON e.dept_id = d.id 
                       GROUP BY 1, 2) t1
               WHERE rank = 1""", sqlite_con)

,name,salary,dept_name,rank
0,Amanda,53000,IT,1
1,Steve,32000,Sales,1


In [23]:
### 2c: Write a query that shows all the employees in each department except the lowest earner. i.e generate the table below
pd.read_sql("""SELECT name, salary, dept_name, rank
               FROM (SELECT name, salary, dept_name, Row_number() OVER (PARTITION BY dept_name ORDER BY salary DESC) rank
                               FROM employee e
                               JOIN department d
                               ON e.dept_id = d.id
                               GROUP BY  1, 3) t1
               WHERE rank != 4
            """, sqlite_con)

,name,salary,dept_name,rank
0,Amanda,53000,IT,1
1,Gafar,41000,IT,2
2,Felix,33000,IT,3
3,Steve,32000,Sales,1
4,Bola,31000,Sales,2
5,Babalola,30200,Sales,3


### Qs 3: Given the students table:

a. Write a query that creates a new column `percentage_score`.  The `percentage_score` is the mean of all the subjects for each student. Round the `percentage_score` to 2 decimal points and sort in descending order.

b. Write a query that selects **ONLY** the students that scored more than the mean of the `percentage_score`.

In [24]:
pd.read_sql("""SELECT *
               FROM students """, sqlite_con)

,matric_no,name,gender,maths,english,physics
0,ux-012-013,Emeka,M,95,84,94
1,ux-012-014,Agnes,F,80,69,88
2,ux-012-015,Alimat,F,69,69,77
3,ux-012-017,Fred,M,28,57,39
4,ux-012-018,Ogechi,F,80,69,78
5,ux-012-019,Steve,M,61,69,72
6,ux-012-016,Esther,F,93,77,91
7,ux-012-020,Ahmed,M,87,94,85
8,ux-012-021,Samantha,F,52,69,48
9,ux-012-022,Philip,M,51,39,45


In [25]:
# 3a.Write a query that creates a new column percentage_score. The percentage_score is the mean of all the subjects for each student.
#  Round the percentage_score to 2 decimal points and sort in descending order.
pd.read_sql("""SELECT matric_no, name, ROUND(AVG(maths + english + physics), 2) percentage_score
               FROM students
               GROUP BY 1,2
               ORDER BY 3 DESC
            """, sqlite_con)

,matric_no,name,percentage_score
0,ux-012-013,Emeka,273.0
1,ux-012-020,Ahmed,266.0
2,ux-012-016,Esther,261.0
3,ux-012-014,Agnes,237.0
4,ux-012-018,Ogechi,227.0
5,ux-012-023,Dave,216.0
6,ux-012-015,Alimat,215.0
7,ux-012-019,Steve,202.0
8,ux-012-024,Seyi,200.0
9,ux-012-021,Samantha,169.0


In [35]:
# 3bi: Write a query that selects ONLY the students that scored more than the mean of the percentage_score.
pd.read_sql("""SELECT AVG(percentage_score)
               FROM (SELECT matric_no, name, ROUND(AVG(maths + english + physics), 2) percentage_score
               FROM students
               GROUP BY 1,2
               ORDER BY 3 DESC) t1
               
               """, sqlite_con)

,AVG(percentage_score)
0,210.416667


In [27]:
# 3bii: Write a query that selects ONLY the students that scored more than the mean of the percentage_score.
pd.read_sql("""SELECT name, SUM(maths + english + physics) total_score
               FROM students
               GROUP BY 1
               HAVING total_score > (SELECT AVG(percentage_score)
                                     FROM (SELECT matric_no, name, ROUND(AVG(maths + english + physics), 2) percentage_score
                                          FROM students
                                          GROUP BY 1,2
                                          ORDER BY 3 DESC) t1)
               
               """, sqlite_con)

,name,total_score
0,Agnes,237
1,Ahmed,266
2,Alimat,215
3,Dave,216
4,Emeka,273
5,Esther,261
6,Ogechi,227
